In [1]:
# Import the required libraries for building the Model
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional


Using TensorFlow backend.


In [2]:
# Reading the source file using pandas 
df = pd.read_excel("tide_file.xlsx")
df

,receipt_id,company_id,matched_transaction_id,feature_transaction_id,DateMappingMatch,AmountMappingMatch,DescriptionMatch,DifferentPredictedTime,TimeMappingMatch,PredictedNameMatch,ShortNameMatch,DifferentPredictedDate,PredictedAmountMatch,PredictedTimeCloseMatch
0,10000,10000,10468,10001,0.00,0.0,0.0,1,0,0.0,0,1,0.0,0
1,10000,10000,10468,10000,0.00,0.0,0.0,1,0,0.0,0,1,0.0,0
2,10000,10000,10468,10003,0.00,0.0,0.0,1,0,0.0,0,1,0.0,0
3,10000,10000,10468,10004,0.00,0.0,0.0,1,0,0.0,0,1,0.0,0
4,10000,10000,10468,10005,0.00,0.0,0.0,1,0,0.0,0,1,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12029,50226,50000,50368,50074,0.65,0.0,0.0,1,0,0.0,0,0,0.0,0
12030,50226,50000,50368,50075,0.65,0.0,0.0,1,0,0.0,0,0,0.0,0
12031,50226,50000,50368,50366,0.00,0.0,0.0,1,0,0.0,0,1,0.0,1
12032,50226,50000,50368,50367,0.00,0.0,0.0,1,0,0.0,0,1,0.0,0


In [3]:
# Creating a 'MATCH_FLAG' based on matching values of feature_transaction_id and matched_transaction_id for all receipts
df['Match_Flag'] = np.where( (df['receipt_id']== df['receipt_id']) & (df['matched_transaction_id']== df['feature_transaction_id']) &  (df['company_id']== df['company_id']) , '1', '0')
df


,receipt_id,company_id,matched_transaction_id,feature_transaction_id,DateMappingMatch,AmountMappingMatch,DescriptionMatch,DifferentPredictedTime,TimeMappingMatch,PredictedNameMatch,ShortNameMatch,DifferentPredictedDate,PredictedAmountMatch,PredictedTimeCloseMatch,Match_Flag
0,10000,10000,10468,10001,0.00,0.0,0.0,1,0,0.0,0,1,0.0,0,0
1,10000,10000,10468,10000,0.00,0.0,0.0,1,0,0.0,0,1,0.0,0,0
2,10000,10000,10468,10003,0.00,0.0,0.0,1,0,0.0,0,1,0.0,0,0
3,10000,10000,10468,10004,0.00,0.0,0.0,1,0,0.0,0,1,0.0,0,0
4,10000,10000,10468,10005,0.00,0.0,0.0,1,0,0.0,0,1,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12029,50226,50000,50368,50074,0.65,0.0,0.0,1,0,0.0,0,0,0.0,0,0
12030,50226,50000,50368,50075,0.65,0.0,0.0,1,0,0.0,0,0,0.0,0,0
12031,50226,50000,50368,50366,0.00,0.0,0.0,1,0,0.0,0,1,0.0,1,0
12032,50226,50000,50368,50367,0.00,0.0,0.0,1,0,0.0,0,1,0.0,0,0


In [4]:
# Declaring the PREDICTORS(Independent features) and RESPONSE(Dependent features) variables 
X = df.iloc[:,0:14]
X


Y = df.iloc[:,-1]
Y


0        0
1        0
2        0
3        0
4        0
        ..
12029    0
12030    0
12031    0
12032    0
12033    1
Name: Match_Flag, Length: 12034, dtype: object

In [0]:
# Splitting the dataset into train set and test set
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2) # 80% training 


In [0]:
# Converting the data into numpy arrays 
x_train = np.array(x_train)
x_test = np.array(x_test)


y_train = np.array(y_train)
y_test = np.array(y_test)



In [7]:
# Verifying the shape of train data set
x_train.shape

(9627, 14)

In [8]:
# Declaring the paramerters of Bi-directional LSTM - RNN Model for learning the matching features
model = Sequential()
model.add(Embedding(80000, 32, input_length=None))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.4))
model.add(Dense(1, activation='sigmoid'))






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
# Compiling the LSTM MODEL using Binary_CrossEntropy
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
# Building the LSTM - RNN model 
print('Train...')

model.fit(x_train, y_train,
          batch_size=100,
          epochs=100,
          validation_data=[x_test, y_test])

Train...



Train on 9627 samples, validate on 2407 samples
Epoch 1/100





9627/9627 [==============================] - 7s 778us/step - loss: 0.3298 - acc: 0.9304 - val_loss: 0.2738 - val_acc: 0.9215
Epoch 2/100
9627/9627 [==============================] - 6s 579us/step - loss: 0.2423 - acc: 0.9306 - val_loss: 0.2721 - val_acc: 0.9215
Epoch 3/100
9627/9627 [==============================] - 6s 572us/step - loss: 0.2015 - acc: 0.9306 - val_loss: 0.4372 - val_acc: 0.9215
Epoch 4/100
9627/9627 [==============================] - 6s 575us/step - loss: 0.1687 - acc: 0.9306 - val_loss: 0.4987 - val_acc: 0.9215
Epoch 5/100
9627/9627 [==============================] - 6s 578us/step - loss: 0.1494 - acc: 0.9312 - val_loss: 0.6330 - val_acc: 0.9202
Epoch 6/100
9627/9627 [==============================] - 6s 577us/step - loss: 0.1340 - acc: 0.9307 - val_loss: 0.7018 - val_acc: 0.9194
Epoch 7/100
9627/9627 [==============================] - 5s 568us/step - loss: 0.1263 - acc: 0.9330 - val_loss: 0

In [11]:
# Verifying the Embedded layers and the Bi-directional parameteres of the model
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 32)          2560000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                16640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 2,576,705
Trainable params: 2,576,705
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Predicting the output response for the test data
predicted_output = model.predict(x_test)

In [16]:
# Verifying the shape of response variables
predicted_output.shape

(2407, 1)

In [18]:
# To Save model and architecture to single file
model.save("model.h5")
print("Saved model to disk")

Saved model to disk


In [20]:
# Verifying the performance of the model on the complete dataset
scores = model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


acc: 97.58%
